**Data Transformation Notebook**

<div style="font-family: system-ui; padding: 20px 30px 20px 20px; background-color: #FFFFFF; border-left: 8px solid #ED9255; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1);max-width:600px;color:#212121;">

- 📅 Date: 14 November 2025
- 👤 Name: Lethokuhle Sikosana
- 🎯 Purpose: Conduct Statistical Analysis on SAPS Crime Data from 2008 - 2013

<span style="display:block;line-height:1.15em;color:#666666;font-size:0.9em;">
</span>

</div>

## Imports

In [8]:
import pandas as pd
import numpy as np

## Loading the Data

In [9]:
crime_data = pd.read_csv('data/sapacr-2008-2023-v1.1.csv', encoding='latin1')
#Check if converted to DataFrame correctly
crime_data.head(3)

,year,station,loc_mn,dc_mn,longitude,latitude,other_theft,arson,assault_gbh,attempted_murder,...,cash_transit_robbery,aggr_robbery,sexual_assault,sexual_offences,police_detected_sexoff,shoplifting,stock_theft,vehicle_theft,theft_from_vehicle,truck_hijacking
0,2008/2009,yeoville,city of johannesburg,city of johannesburg,28.06281,-26.1829,491,3,570,25,...,1,293,7,119,0,218,0,143,272,0
1,2009/2010,yeoville,city of johannesburg,city of johannesburg,28.06281,-26.1829,452,5,625,9,...,1,276,13,115,0,185,0,124,300,0
2,2010/2011,yeoville,city of johannesburg,city of johannesburg,28.06281,-26.1829,525,1,586,15,...,0,238,23,101,0,164,0,96,256,0


## Data Transformations

### Creating Crime Categories and Totalling Them

In [10]:
#Creating columns using the SAPS defined crime categories
crime_data['contact_related_crime'] = crime_data['arson'] + crime_data['malicious_damage']

crime_data['crimes_against_property'] = (
    crime_data[['burglary_res', 'burglary_nonres', 'vehicle_theft', 'theft_from_vehicle', 'stock_theft']].sum(axis=1)
)

crime_data['contact_crime'] = (
    crime_data[['murder', 'attempted_murder', 'sexual_offences', 'assault_gbh', 'common_assault', 'common_robbery', 'aggr_robbery']].sum(axis=1)
)

crime_data['other_serious_crimes'] = (
    crime_data[['other_theft', 'commercial_crime', 'shoplifting']].sum(axis=1)
)

crime_data['crimes_dependent_on_police_action_for_detention'] = (
    crime_data[['illegal_firearms', 'dui', 'drug_crime', 'police_detected_sexoff']].sum(axis=1)
)

#Creating a total crime column based on the sum of these categories (this is citizen reported crime)
crime_data['total_crime'] = crime_data[
    ['crimes_against_property', 'contact_crime', 'other_serious_crimes', 'crimes_dependent_on_police_action_for_detention']
].sum(axis=1)

#Creating a total crime excluding police dependent reports (essentially a measure of police activity)
crime_data['total_crime_excl_police'] = (
    crime_data['total_crime'] - crime_data['crimes_dependent_on_police_action_for_detention']
)

#Check if transformed DataFrame is correct
crime_data.head(3)

,year,station,loc_mn,dc_mn,longitude,latitude,other_theft,arson,assault_gbh,attempted_murder,...,vehicle_theft,theft_from_vehicle,truck_hijacking,contact_related_crime,crimes_against_property,contact_crime,other_serious_crimes,crimes_dependent_on_police_action_for_detention,total_crime,total_crime_excl_police
0,2008/2009,yeoville,city of johannesburg,city of johannesburg,28.06281,-26.1829,491,3,570,25,...,143,272,0,239,745,1606,870,360,3581,3221
1,2009/2010,yeoville,city of johannesburg,city of johannesburg,28.06281,-26.1829,452,5,625,9,...,124,300,0,257,690,1639,791,569,3689,3120
2,2010/2011,yeoville,city of johannesburg,city of johannesburg,28.06281,-26.1829,525,1,586,15,...,96,256,0,209,666,1543,936,346,3491,3145


### Creating a New DataFrame with Desired Information and Format

In [11]:
#Create a new DataFrame with just data from Rustenburg Municipality
rustenburg_municipality_df = (
    crime_data[crime_data['loc_mn'].isin(['rustenburg'])]
    .groupby(['year', 'station'])[
        [
            'total_crime',
            'contact_related_crime',
            'crimes_against_property',
            'other_serious_crimes',
            'contact_crime',
            'crimes_dependent_on_police_action_for_detention',
            'total_crime_excl_police'
        ]
    ].sum()
    .reset_index()
)

rustenburg_municipality_df.head(5)

,year,station,total_crime,contact_related_crime,crimes_against_property,other_serious_crimes,contact_crime,crimes_dependent_on_police_action_for_detention,total_crime_excl_police
0,2008/2009,bethanie,1129,82,249,265,550,65,1064
1,2008/2009,boitekong,1666,98,411,149,741,365,1301
2,2008/2009,boons,244,15,88,81,60,15,229
3,2008/2009,lethabong,573,34,131,68,227,147,426
4,2008/2009,marikana,1640,119,354,451,657,178,1462


### Adding the Columns Needed for Statistical Inference

In [12]:
#Transforming crime category columns to logs 
#Adding 1 in case values are 0 (log of zero is undefined)
rustenburg_municipality_df['crimes_against_property_log'] = np.log(
    rustenburg_municipality_df['crimes_against_property'] + 1
)
rustenburg_municipality_df['contact_crime_log'] = np.log(
    rustenburg_municipality_df['contact_crime'] + 1
)
rustenburg_municipality_df['contact_related_crime_log'] = np.log(
    rustenburg_municipality_df['contact_related_crime'] + 1
)
rustenburg_municipality_df['other_serious_crimes_log'] = np.log(
    rustenburg_municipality_df['other_serious_crimes'] + 1
)
rustenburg_municipality_df['crimes_dependent_on_police_action_for_detention_log'] = np.log(
    rustenburg_municipality_df['crimes_dependent_on_police_action_for_detention'] + 1
)
rustenburg_municipality_df['total_crime_log'] = np.log(
    rustenburg_municipality_df['total_crime'] + 1
)
rustenburg_municipality_df['total_crime_excl_police_log'] = np.log(rustenburg_municipality_df['total_crime_excl_police'] + 1)


## Creating the Columns for the DiD Model

In [13]:
# Define treatment (Marikana = 1, Rustenburg = 0)
rustenburg_municipality_df['treated'] = (rustenburg_municipality_df['station'] == 'marikana').astype(int)

# Convert year to numeric (take first 4 digits)
rustenburg_municipality_df['year_numeric'] = rustenburg_municipality_df['year'].str[:4].astype(int)

# Define post-treatment indicator (year 2012/2013 or later)
rustenburg_municipality_df['post'] = (
    (rustenburg_municipality_df['year_numeric'] >= 2012) &
    (rustenburg_municipality_df['year_numeric'] <= 2015)
).astype(int)

# Interaction term (DiD)
rustenburg_municipality_df['did'] = rustenburg_municipality_df['treated'] * rustenburg_municipality_df['post']

rustenburg_municipality_df.head(5)

,year,station,total_crime,contact_related_crime,crimes_against_property,other_serious_crimes,contact_crime,crimes_dependent_on_police_action_for_detention,total_crime_excl_police,crimes_against_property_log,contact_crime_log,contact_related_crime_log,other_serious_crimes_log,crimes_dependent_on_police_action_for_detention_log,total_crime_log,total_crime_excl_police_log,treated,year_numeric,post,did
0,2008/2009,bethanie,1129,82,249,265,550,65,1064,5.521461,6.311735,4.418841,5.583496,4.189655,7.029973,6.970730,0,2008,0,0
1,2008/2009,boitekong,1666,98,411,149,741,365,1301,6.021023,6.609349,4.595120,5.010635,5.902633,7.418781,7.171657,0,2008,0,0
2,2008/2009,boons,244,15,88,81,60,15,229,4.488636,4.110874,2.772589,4.406719,2.772589,5.501258,5.438079,0,2008,0,0
3,2008/2009,lethabong,573,34,131,68,227,147,426,4.882802,5.429346,3.555348,4.234107,4.997212,6.352629,6.056784,0,2008,0,0
4,2008/2009,marikana,1640,119,354,451,657,178,1462,5.872118,6.489205,4.787492,6.113682,5.187386,7.403061,7.288244,1,2008,0,0


## Saving the New DataFrame for Other's Replication

In [14]:
#Save as a CSV for easier analysis later
rustenburg_municipality_df.to_csv('data/rustenburg_municipality_crime_data.csv', 
    index=False)